In [1]:
import netCDF4
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize

In [ ]:
famine_nc_file = 'Demographics/gpw-v4-population-density-rev11_totpop_15_min_nc/gpw_v4_population_density_rev11_15_min.nc'
nc = netCDF4.Dataset(famine_nc_file, mode='r')

In [ ]:
nc.variables.keys()

In [ ]:
nc.variables

In [ ]:
lat = nc.variables['latitude'][:]
lon = nc.variables['longitude'][:]
time_var = nc.variables['time']

In [ ]:
dtime = netCDF4.num2date(time_var[:],time_var.units)
precip = nc.variables['rfe'][:]

In [ ]:
precip.shape

In [ ]:
precipnan = np.ma.filled(precip.astype(float), np.nan)

In [ ]:
precipnan.shape

In [ ]:
precip = precip.transpose(2,0,1).reshape(1,-1)

In [ ]:
precip.shape

In [ ]:
tempseries = pd.Series(precip, index=dtime) 


In [ ]:
temp40nan.shape

In [ ]:
import xarray as xr

nc = xr.open_dataset('Demographics/gpw-v4-population-density-rev11_totpop_15_min_nc/gpw_v4_population_density_rev11_15_min.nc')
nc = nc.rename({'Population Density, v4.11 (2000, 2005, 2010, 2015, 2020): 15 arc-minutes':'PopulationDensity'})
df = nc.PopulationDensity.to_dataframe()
df.dropna().to_csv('Demographics/popdensity.csv')

In [ ]:
import os
import xarray as xr
for filename in os.listdir('Soil Moisture'):
    nc = xr.open_dataset('Soil Moisture/'+filename)
    nc.SoilMoi00_10cm_tavg.to_dataframe().to_csv('Soil Moisture/'+filename+'.csv')


In [2]:
ea = pd.read_csv('Soil Moisture/FLDAS_NOAH01_A_EA_M.A201908.001.nc.csv').dropna()
sa = pd.read_csv('Soil Moisture/FLDAS_NOAH01_A_SA_M.A201908.001.nc.csv').dropna()
wa = pd.read_csv('Soil Moisture/FLDAS_NOAH01_A_WA_M.A201908.001.nc.csv').dropna()

In [3]:
soilMoisture = pd.concat([ea,sa,wa])

In [4]:
rain = pd.read_csv('Rainfall Estimates/2019/rfe2019_09.v3.nc.csv',index_col=False).dropna()

In [ ]:
rain.head()

In [ ]:
soilMoisture.head()

In [5]:
soilMoisture = soilMoisture.rename(columns={"Y": "lat", "X": "lon"})

In [6]:
soilMoisture = soilMoisture.rename(columns={"SoilMoi00_10cm_tavg": "Soil Moisture 10cm"})

In [ ]:
soilMoisture.head()

In [ ]:
os.listdir('Rainfall Estimates/2019')

In [7]:
rain = pd.read_csv('Rainfall Estimates/2019/rfe2019_09.v3.nc.csv').dropna()

In [ ]:
rain.head()

In [8]:
veg = pd.read_csv('Vegetation/vegetation.csv').dropna()

In [ ]:
veg.head()

In [9]:
veg = veg.rename(columns={"latitude": "lat", "longitude": "lon"})
veg.head()


,time,lat,lon,NDVI
0,2019-10-17,76.125,-102.824997,-0.0415
1,2019-10-17,76.125,-102.774994,-0.0415
2,2019-10-17,76.125,-102.724998,-0.0415
3,2019-10-17,76.125,-102.674995,-0.0335
4,2019-10-17,76.125,-102.625000,-0.0256


In [10]:
lat_values = rain.lat.values
lon_values = rain.lon.values

In [ ]:
lon_values.min()

In [ ]:
veg_Africa = veg.loc[((veg['lon'] >= lon_values.min())&(veg['lon'] <= lon_values.max())) & ((veg['lat'] >= lat_values.min()) & (veg['lat'] <= lat_values.max()))]

In [ ]:
veg_Africa.shape

In [ ]:
veg_Africa

In [ ]:
import pandas as pd
pd.merge(df1, df2, on="lat")

In [ ]:
import os
import glob
import pandas as pd
os.chdir('Vegetation')

In [ ]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "rfe2019.v3.nc.csv", index=False, encoding='utf-8-sig')

In [12]:
import pandas
import json
#df = pandas.read_csv('vegetation.csv')


In [ ]:
rain = rain.drop('level_0',axis=1)
soilMoisture = soilMoisture.drop('level_0',axis=1)
veg = veg.drop('level_0',axis=1)

In [13]:
rain = rain.drop('time',axis=1)
soilMoisture = soilMoisture.drop('time',axis=1)
veg = veg.drop('time',axis=1)

In [14]:
pop = pd.read_csv('Demographics/popdensity.csv').dropna()

In [15]:
pop = pop.rename(columns={"latitude": "lat", "longitude": "lon"})
pop = pop.drop('raster',axis=1)

In [16]:
pop.head()

,lat,lon,PopulationDensity
0,83.625,-37.875,0.0
1,83.625,-37.625,0.0
2,83.625,-36.875,0.0
3,83.625,-36.625,0.0
4,83.625,-36.375,0.0


In [16]:
pop_Africa = pop.loc[((pop['lon'] >= lon_values.min())&(pop['lon'] <= lon_values.max())) & ((pop['lat'] >= lat_values.min()) & (pop['lat'] <= lat_values.max()))]
#pop_Africa.to_csv('Demographics/AfricaDensity.csv', index = False)

In [17]:
rain = rain.reset_index(drop=True)
soilMoisture = soilMoisture.reset_index(drop=True)
veg = veg.reset_index(drop=True)
pop = pop_Africa.reset_index(drop=True)

In [18]:
rain.head()

,lat,lon,rfe
0,38.025,-8.8500,9.0
1,38.025,-8.8125,9.0
2,38.025,-8.7750,10.0
3,38.025,-8.7375,17.0
4,38.025,-8.7000,17.0


In [19]:
full_data = pd.concat([rain,soilMoisture,veg, pop],axis=0)

c:\programdata\anaconda3\envs\work\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [20]:
full_data.head()

,NDVI,PopulationDensity,Soil Moisture 10cm,lat,lon,rfe
0,NaN,NaN,NaN,38.025,-8.8500,9.0
1,NaN,NaN,NaN,38.025,-8.8125,9.0
2,NaN,NaN,NaN,38.025,-8.7750,10.0
3,NaN,NaN,NaN,38.025,-8.7375,17.0
4,NaN,NaN,NaN,38.025,-8.7000,17.0


In [21]:
full_data_interpolated = full_data.interpolate(method = 'cubic',axis = 0)

In [57]:
full_data_interpolated.dropna().shape

(235253, 6)

In [ ]:
full_data_interpolated.dropna(inplace=True)

In [22]:
final_data = full_data_interpolated.rename(columns= {'rfe':'Rainfall (mm)', 'PopulationDensity': 'Population Density'}).dropna()

In [24]:
final_data.head()

,NDVI,Population Density,Soil Moisture 10cm,lat,lon,Rainfall (mm)
0,-0.0415,85.231030,0.070788,37.875,-8.875,9.0
1,-0.0415,11.993163,0.062295,37.875,-8.625,9.0
2,-0.0415,7.142940,0.071134,37.875,-8.375,10.0
3,-0.0335,20.873466,0.071186,37.875,-8.125,17.0
4,-0.0256,10.336887,0.071502,37.875,-7.875,17.0


In [ ]:
len(final_data)

In [29]:
final_data

,NDVI,Population Density,Soil Moisture 10cm,lat,lon,Rainfall (mm)
0,-0.0415,85.231030,0.070788,37.875,-8.875,9.0
1,-0.0415,11.993163,0.062295,37.875,-8.625,9.0
2,-0.0415,7.142940,0.071134,37.875,-8.375,10.0
3,-0.0335,20.873466,0.071186,37.875,-8.125,17.0
4,-0.0256,10.336887,0.071502,37.875,-7.875,17.0
5,-0.0228,11.434233,0.062946,37.875,-7.625,16.0
6,-0.0228,11.270674,0.073858,37.875,-7.375,14.0
7,-0.0200,7.868647,0.075550,37.875,-7.125,15.0
8,-0.0200,11.174693,0.077354,37.875,-6.875,16.0
9,-0.0304,22.116201,0.080662,37.875,-6.625,13.0


In [88]:
normalized_df=(final_data-final_data.mean())/final_data.std()

In [89]:
normalized_df.loc[normalized_df['Rainfall (mm)'] == 0]

,NDVI,Population Density,Soil Moisture 10cm,lat,lon,Rainfall (mm),FRI


In [34]:
import math

In [ ]:
for i in range(len(final_data)):
    if i == 0:
        print(i)

In [32]:
final_data.head()

,NDVI,Population Density,Soil Moisture 10cm,lat,lon,Rainfall (mm),FRI
0,-0.0415,85.231030,0.070788,37.875,-8.875,9.0,NaN
1,-0.0415,11.993163,0.062295,37.875,-8.625,9.0,NaN
2,-0.0415,7.142940,0.071134,37.875,-8.375,10.0,NaN
3,-0.0335,20.873466,0.071186,37.875,-8.125,17.0,NaN
4,-0.0256,10.336887,0.071502,37.875,-7.875,17.0,NaN


In [ ]:
remove that code its extremely slow, use this vectorized form. and whyis it NDVI instead of data[NDVI]

In [86]:
final_data['FRI'] = FRI_norm

In [84]:
FRI_abs = np.absolute(FRI)

In [91]:
FRI_norm = (FRI_abs - FRI_abs.min())/(FRI_abs.max()-FRI_abs.min())

c:\programdata\anaconda3\envs\work\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [90]:
FRI = np.array([(np.cbrt((normalized_df["Population Density"][i]))/(np.absolute((normalized_df["Rainfall (mm)"][i])) * normalized_df["NDVI"][i])) for i in range(len(normalized_df))]) # this should work just fix the pandas query

In [87]:
final_data.loc[final_data['FRI'] == final_data['FRI'].max()]

,NDVI,Population Density,Soil Moisture 10cm,lat,lon,Rainfall (mm),FRI
0,-0.0415,85.231030,0.070788,37.875,-8.875,9.0,0.0
1,-0.0415,11.993163,0.062295,37.875,-8.625,9.0,0.0
2,-0.0415,7.142940,0.071134,37.875,-8.375,10.0,0.0
3,-0.0335,20.873466,0.071186,37.875,-8.125,17.0,0.0
4,-0.0256,10.336887,0.071502,37.875,-7.875,17.0,0.0
5,-0.0228,11.434233,0.062946,37.875,-7.625,16.0,0.0
6,-0.0228,11.270674,0.073858,37.875,-7.375,14.0,0.0
7,-0.0200,7.868647,0.075550,37.875,-7.125,15.0,0.0
8,-0.0200,11.174693,0.077354,37.875,-6.875,16.0,0.0
9,-0.0304,22.116201,0.080662,37.875,-6.625,13.0,0.0


In [40]:
len(FRI)

235253

In [120]:
import json

In [77]:
final_data

,NDVI,Population Density,Soil Moisture 10cm,lat,lon,Rainfall (mm),FRI
0,-0.0415,85.231030,0.070788,37.875,-8.875,9.0,0.000393
1,-0.0415,11.993163,0.062295,37.875,-8.625,9.0,0.000389
2,-0.0415,7.142940,0.071134,37.875,-8.375,10.0,0.000888
3,-0.0335,20.873466,0.071186,37.875,-8.125,17.0,0.000045
4,-0.0256,10.336887,0.071502,37.875,-7.875,17.0,0.000078
5,-0.0228,11.434233,0.062946,37.875,-7.625,16.0,0.000112
6,-0.0228,11.270674,0.073858,37.875,-7.375,14.0,0.000164
7,-0.0200,7.868647,0.075550,37.875,-7.125,15.0,0.000182
8,-0.0200,11.174693,0.077354,37.875,-6.875,16.0,0.000149
9,-0.0304,22.116201,0.080662,37.875,-6.625,13.0,0.000114


In [103]:
json_result_string = final_data.to_json(
    orient='records', 
    double_precision=12,
)
json_result = json.loads(json_result_string)

In [104]:
geojson = {
    'type': 'FeatureCollection',
    'features': []
}
for record in json_result:
    geojson['features'].append({
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [record['lat'], record['lon']],
        },
        'properties': record,
    })

In [105]:
with open('Final Data/final_data.json', 'w') as f:
    f.write(json.dumps(geojson, indent=2))

In [106]:
final_data.to_csv('Final Data/final_data.csv', index=False)

In [ ]:
export as CSV too please because we need CSV got it